In [2]:
from google.colab import drive
from google.colab import files
import os
import seaborn as sns
import pandas as pd
from sklearn.metrics import confusion_matrix
import torchvision.models as models
import numpy as np
import matplotlib.pyplot as plt
import time
import torchvision
from torchsummary import summary
import torchvision.transforms as transforms
import torch.nn as nn
from PIL import Image as PilImage
from torchvision import models, transforms
import torch
import json
import plotly.io as pio
pio.renderers.default = "png"


In [3]:
num_classes=5 #based on dataset change the values 5,15,35
# Change the model to use model of your choice from resnet18,resne34,mobileNetv2
model = models.resnet18(pretrained=False, num_classes=num_classes)
# navigate to Output/{Model_Selected}/Weights folder choose the  model weight file based on the dataset you want to test against
model.load_state_dict(torch.load('/content/5-class-100-epochos-output.pt'))  # This is using model trained for 100 epochos on dataset 1 with 5 classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #this will use GPU if avilable
model.to(device)


c:\Users\ozwin\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ozwin\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 162, 162]           9,408
       BatchNorm2d-2         [-1, 64, 162, 162]             128
              ReLU-3         [-1, 64, 162, 162]               0
         MaxPool2d-4           [-1, 64, 81, 81]               0
            Conv2d-5           [-1, 64, 81, 81]          36,864
       BatchNorm2d-6           [-1, 64, 81, 81]             128
              ReLU-7           [-1, 64, 81, 81]               0
            Conv2d-8           [-1, 64, 81, 81]          36,864
       BatchNorm2d-9           [-1, 64, 81, 81]             128
             ReLU-10           [-1, 64, 81, 81]               0
       BasicBlock-11           [-1, 64, 81, 81]               0
           Conv2d-12           [-1, 64, 81, 81]          36,864
      BatchNorm2d-13           [-1, 64, 81, 81]             128
             ReLU-14           [-1, 64,

In [ ]:
img_fath = "/content/5-class/preprocessed-cleaned-set/test/class-4/0d32920975d1d318089d4b17920669e4.jpg" #specify the image path

In [ ]:
def predict(model):
    def transform_image():
        # for dataset 1 use mean=[0.3461, 0.3217, 0.2852], std=[0.2870, 0.2718, 0.2510]
        # for dataset 2 use mean=[] std=[] need to calculate this based on the dataset
        # for dataset 3 use mean=[] std=[] need to calculate this based on the dataset
        transform = transforms.Compose([transforms.Resize(255),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize(mean=[0.3461, 0.3217, 0.2852], std=[0.2870, 0.2718, 0.2510] 
                                                )])
        image = PilImage.open(img_fath)
        return transform(image).unsqueeze(0) #for single image

    model.eval()
    
    labels=[]  #specify the labels of the dataset selected
    for i in range(num_classes):
        labels+=f'class-{i}'

    def get_prediction():
        tensor = transform_image()
        tensor=tensor.to(device)
        output = model(tensor)
        probs = torch.nn.functional.softmax(output, dim=1)
        print(probs.cpu().detach().numpy())
        conf, classes = torch.max(probs, 1)
        return conf.item(), labels[classes.item()],classes.item()

    image_path = img_fath
    image = plt.imread(image_path)
    plt.imshow(image)

    with open(image_path, 'rb') as f:
    
        conf,y_pre,index=get_prediction()
        print(y_pre, ' at confidence score:{0:.2f}'.format(conf))
        return (y_pre, ' at confidence score:{0:.2f}'.format(conf))

In [ ]:
output, index = predict(model) 
print(output)